In [87]:
from PIL import Image
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from pathlib import Path


def create_chart(
        csv_filename: str,
        date_column: str,
        chart_type: str,
        columns_to_plot: dict,
        data_folder: str = "data",
        watermark: str = "off",
        title: str = None
) -> go.Figure:
    """
    Create a chart from CSV data with specified parameters.
    
    Parameters:
    -----------
    csv_filename : str
        Name of the CSV file (e.g., 'lending_market_rate.csv')
    date_column : str
        Name of the column containing dates to be used as index
    chart_type : str
        Type of chart to create ('line', 'bar', or 'area')
    columns_to_plot : dict
        Dictionary mapping column names to display names 
        (e.g., {'fed_rate': 'Federal Rate', 'defi_rate': 'DeFi Rate'})
    data_folder : str
        Name of the folder containing the CSV files (default: 'data')
    watermark : str
        Whether to add watermark and styling ('on' or 'off', default: 'off')
    title : str
        Title text for the chart (optional)
    """
    # Try different path options
    possible_paths = [
        Path.cwd() / data_folder / csv_filename,
        Path.cwd().parent / data_folder / csv_filename,
        Path(data_folder) / csv_filename
    ]

    # Print current working directory for debugging
    print(f"Current working directory: {Path.cwd()}")

    # Try each path until we find one that works
    df = None
    for path in possible_paths:
        try:
            if path.exists():
                df = pd.read_csv(path)
                print(f"Successfully loaded data from: {path}")
                break
        except Exception as e:
            print(f"Tried path: {path} (not found)")
            continue

    if df is None:
        raise FileNotFoundError(f"Could not find {csv_filename} in any of these locations:\n" +
                                "\n".join(str(p) for p in possible_paths))

    # Convert date column to datetime
    df[date_column] = pd.to_datetime(df[date_column])

    # Create figure
    fig = go.Figure()

    # Add traces based on chart type
    for column, display_name in columns_to_plot.items():
        if chart_type.lower() == 'line':
            fig.add_trace(
                go.Scatter(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    mode='lines'
                )
            )

        elif chart_type.lower() == 'bar':
            fig.add_trace(
                go.Bar(
                    x=df[date_column],
                    y=df[column],
                    name=display_name
                )
            )

        elif chart_type.lower() == 'area':
            fig.add_trace(
                go.Scatter(
                    x=df[date_column],
                    y=df[column],
                    name=display_name,
                    mode='lines',
                    stackgroup='one',
                    fill='tonexty'
                )
            )
        else:
            raise ValueError("Chart type must be 'line', 'bar', or 'area'")

    # Get min and max values for y-axis
    y_min = min(df[column].min() for column in columns_to_plot.keys())
    y_max = max(df[column].max() for column in columns_to_plot.keys())

    # Calculate font sizes
    axis_size = 17

    # Define colors - darker grey
    border_color = '#7f7f7f'

    # Set default title text if not provided
    default_title = f'{chart_type.capitalize()} Chart: {", ".join(columns_to_plot.values())}'

    # Basic layout update with new styling
    fig.update_layout(
        title=dict(
            text=title if title else default_title,
            font=dict(
                color='#000000',
                size=29,
                weight='bold'
            ),
            x=0.055,
            y=0.94,
            xanchor='left',
            yanchor='top',
            pad=dict(t=0, b=0)
        ),
        width=1350,
        height=750,
        showlegend=True,
        font=dict(color=border_color),
        # Legend inside top left of chart
        legend=dict(
            orientation="h",
            yanchor="top",
            y=0.98,
            xanchor="left",
            x=0.02,
            font=dict(
                color=border_color,
                size=axis_size
            ),
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        # Axis formatting
        xaxis=dict(
            title=None,
            tickformat="%b '%y",
            dtick="M3",
            tickangle=0,
            tickmode='array',
            ticktext=[
                date.strftime("%b '%y")
                for date in pd.date_range(
                    start=df[date_column].min(),
                    end=df[date_column].max(),
                    freq='3ME'
                )
            ],
            tickvals=[
                date
                for date in pd.date_range(
                    start=df[date_column].min(),
                    end=df[date_column].max(),
                    freq='3ME'
                )
            ],
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            showgrid=False,
            ticks='outside',
            ticklen=8,
            tickwidth=1
        ),
        yaxis=dict(
            title=None,
            showgrid=False,
            tickfont=dict(
                color=border_color,
                size=axis_size
            ),
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[y_min, y_max],
            tickmode='auto',
            nticks=8,
            mirror=True,
            side='left',
            showticksuffix='none',
            showtickprefix='none',
            showticklabels=True,
            tickson='labels'
        )
    )

    # Add watermark and styling if requested
    if watermark.lower() == "on":
        try:
            # Open and add the watermark image
            watermark_img = Image.open("glassnode_large.png")
            fig.add_layout_image(
                dict(
                    source=watermark_img,
                    sizex=0.36,
                    sizey=0.36,
                    xanchor="center",
                    yanchor="middle",
                    xref="paper",
                    yref="paper",
                    x=0.5,
                    y=0.5,
                    opacity=0.15,
                    layer="above"
                )
            )

            # Add copyright text
            fig.add_annotation(
                showarrow=False,
                text=f"© {str(datetime.today().year)} Glassnode. All Rights Reserved",
                font=dict(
                    size=15,
                    color=border_color
                ),
                xref='paper',
                yref='paper',
                x=1,
                y=-0.125,
                opacity=0.5
            )
        except FileNotFoundError:
            print("Warning: Watermark image not found. Skipping watermark but applying styling.")

        # Apply styling regardless of whether watermark image was found
        fig.update_layout({
            'plot_bgcolor': 'white',
            'paper_bgcolor': 'white',
            'xaxis': {
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'showgrid': False,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1
            },
            'yaxis': {
                'showgrid': False,
                'linecolor': border_color,
                'linewidth': 1,
                'mirror': True,
                'ticks': 'outside',
                'ticklen': 8,
                'tickwidth': 1,
                'range': [y_min, y_max],
                'side': 'left',
                'showticksuffix': 'none',
                'showtickprefix': 'none',
                'showticklabels': True,
                'tickson': 'labels'
            },
            'autosize': True
        })

    return fig


# Example usage with simplified title input
fig = create_chart(
    csv_filename='lending_market_rate.csv',
    date_column='t',
    chart_type='line',
    columns_to_plot={
        'fed_rate': 'Federal Rate',
        'defi_rate': 'DeFi Lending Rate'
    },
    watermark="on",
    title='Lending Market Rates Comparison'  # Simple string input for title
)

# Show the plot
fig.show()

Current working directory: E:\Projects\Glassnode\projects\glassnode_chart\fasanara
Successfully loaded data from: E:\Projects\Glassnode\projects\glassnode_chart\data\lending_market_rate.csv
